In [ ]:
from bs4 import BeautifulSoup as soup
import requests
import re
import pandas as pd
from selenium import webdriver
import html5lib
import numpy as np
import csv
from collections import defaultdict
from tabulate import tabulate
import openpyxl
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning



In [ ]:
HEADER = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
#PROXY = {\"https\": \"https//59.110.7.190:1080\"}"

In [ ]:
baseURL = 'https://www.civivi.com'
product_links = []
for x in range (1,7):
    HTML = requests.get(f'https://www.civivi.com/collections/all-products/price-range_-70?page={x}',HEADER)
    time.sleep(1)
    #HTML.status_code
    Booti= soup(HTML.content, "lxml")
    knife_items = Booti.find_all('div',class_= "product-list product-list--collection product-list--with-sidebar")
    
    for items in knife_items:
        for links in items.findAll('a', attrs = {'class' : 'product-item__image-wrapper product-item__image-wrapper--with-secondary'}, href = True):
            product_links.append(baseURL + links['href'])
    

In [ ]:
len(product_links)

In [ ]:
# dataframe that will hold the final resulting data
final = pd.DataFrame()
for links in product_links:
    requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
    HTML2 = requests.get(links, HEADER, verify=False)
    
    time.sleep(1)
    Booti2 = soup(HTML2.content,"lxml")
    try:
        buffer = pd.DataFrame(
            [[
                # name
                Booti2.find('h1', class_='product-meta__title heading h1').text.strip(),
                # price
                Booti2.find('div', class_='price-list').find('span').text,
                # if  you don't want $ do this: Booti2.find('div', class_='price-list').find('span').text[1:]
                # Weight
                str(Booti2.find('div', class_ = 'rte text--pull').find(text=re.compile(r'(?:Blade Finish: )'))[14:]).replace('Blade Finish: ', '')
                
            ]],
            columns=['Name', 'Price', 'Weight']
        )
        final = final.append(buffer)
    except:
        continue
print(final)
print(len(final))

In [ ]:
final.to_excel("civivivivivi.xlsx")